In [20]:
import cv2
import numpy as np

def compute_homography(src_pts, dst_pts):
    """
    计算单应矩阵 H。
    
    :param src_pts: 源点坐标
    :param dst_pts: 目标点坐标
    :return: 单应矩阵 H
    """
    A = []

    for i in range(len(src_pts)):
        x, y = src_pts[i][0], src_pts[i][1]
        u, v = dst_pts[i][0], dst_pts[i][1]
        A.append([-x, -y, -1, 0, 0, 0, x * u, y * u, u])
        A.append([0, 0, 0, -x, -y, -1, x * v, y * v, v])

    A = np.array(A)
    U, S, V = np.linalg.svd(A)
    H = V[-1].reshape(3, 3)
    H = H / H[2, 2]

    return H

def ransac_homography(src_pts, dst_pts, threshold=5.0, max_iter=2000, confidence=0.99):
    """
    使用RANSAC估计单应矩阵。
    
    :param src_pts: 源点坐标
    :param dst_pts: 目标点坐标
    :param threshold: 内点距离阈值
    :param max_iter: 最大迭代次数
    :param confidence: 置信度
    :return: 最佳单应矩阵 H 和内点掩码
    """
    max_inliers = []
    best_H = None

    for i in range(max_iter):
        # 随机选择4个点
        idx = np.random.choice(len(src_pts), 4, replace=False)
        # 计算单应矩阵
        H = compute_homography(src_pts[idx], dst_pts[idx])

        # 变换源点
        src_pts_homogeneous = np.concatenate((src_pts, np.ones((len(src_pts), 1))), axis=1)
        src_pts_transformed = np.dot(H, src_pts_homogeneous.T).T
        src_pts_transformed = src_pts_transformed[:, :2] / src_pts_transformed[:, 2:]

        # 计算残差
        errors = np.linalg.norm(dst_pts - src_pts_transformed, axis=1)
        inliers = errors < threshold

        # 更新最佳单应矩阵和内点数量
        if np.sum(inliers) > np.sum(max_inliers):
            max_inliers = inliers
            best_H = H

        # 判断是否达到置信度
        if np.sum(inliers) / len(src_pts) > confidence:
            break

        # 显示 RANSAC 迭代过程中的信息
        if (i + 1) % 100 == 0:
            print(f"RANSAC iteration {i + 1}: Inliers ratio = {np.sum(inliers) / len(src_pts):.2f}")

    return best_H, max_inliers

def Panorama_stitching(image_right, image_left, downscale_factor=2):
    """
    全景拼接函数。
    
    :param image_right: 右侧图像
    :param image_left: 左侧图像
    :param downscale_factor: 下采样因子
    :return: 拼接后的全景图像
    """
    # 对图像进行下采样
    image_right_downscaled = cv2.resize(image_right, None, fx=1/downscale_factor, fy=1/downscale_factor)
    image_left_downscaled = cv2.resize(image_left, None, fx=1/downscale_factor, fy=1/downscale_factor)

    # 转换为灰度图像
    gray_right = cv2.cvtColor(image_right_downscaled, cv2.COLOR_BGR2GRAY)
    gray_left = cv2.cvtColor(image_left_downscaled, cv2.COLOR_BGR2GRAY)

    # 使用SIFT特征检测器
    sift = cv2.SIFT_create()
    keypoints_right, descriptors_right = sift.detectAndCompute(gray_right, None)
    keypoints_left, descriptors_left = sift.detectAndCompute(gray_left, None)

    # 使用暴力匹配器进行特征匹配
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors_right, descriptors_left, k=2)

    # 通过Lowe's ratio test筛选出好的匹配点
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        src_pts = np.float32([keypoints_right[m.queryIdx].pt for m in good_matches]) * downscale_factor
        dst_pts = np.float32([keypoints_left[m.trainIdx].pt for m in good_matches]) * downscale_factor

        # 使用RANSAC计算单应矩阵
        H, mask = ransac_homography(src_pts, dst_pts)
        matches_mask = np.array(mask, dtype=int).tolist()  # 转换为Int类型列表

        # 图像变形
        h_right, w_right = image_right.shape[:2]
        h_left, w_left = image_left.shape[:2]
        panorama = cv2.warpPerspective(image_right, H, (w_right + w_left, max(h_right, h_left)))
        panorama[0:h_left, 0:w_left] = image_left

        # 显示单应矩阵
        print("Homography matrix:")
        print(H)

        # 显示匹配点比例
        print(f"Matches ratio: {len(good_matches) / len(matches):.2f}")

        # 显示匹配点
        draw_params = dict(matchColor=(0, 255, 0),  # 在匹配的关键点间画绿线
                           singlePointColor=None,
                           matchesMask=matches_mask,  # 只画内点
                           flags=2)
        img_matches = cv2.drawMatches(image_right_downscaled, keypoints_right, image_left_downscaled, keypoints_left, good_matches, None, **draw_params)

        cv2.imshow("Matches", img_matches)
        cv2.imwrite('matches.jpg', img_matches)

        return panorama

    else:
        print("Not enough matches are found - {}/{}".format(len(good_matches), 10))
        return None

# 主程序
if __name__ == "__main__":
    # 读取左右图像
    image_right = cv2.imread('right.jpg')
    image_left = cv2.imread('left.jpg')

    # 调用全景拼接函数，得到全景图像
    panorama = Panorama_stitching(image_right, image_left)

    # 判断全景图像是否为空，如果不为空则显示并保存全景图像
    if panorama is not None:
        # 显示全景图像
        cv2.imshow('Panorama', panorama)
        cv2.imwrite('panorama.jpg', panorama)  # 保存全景图像
        cv2.waitKey(0)
    else:
        print("Panorama stitching failed.")

    cv2.destroyAllWindows()

RANSAC iteration 100: Inliers ratio = 0.76
RANSAC iteration 200: Inliers ratio = 0.41
RANSAC iteration 300: Inliers ratio = 0.00
RANSAC iteration 400: Inliers ratio = 0.00
RANSAC iteration 500: Inliers ratio = 0.41
RANSAC iteration 600: Inliers ratio = 0.00
RANSAC iteration 700: Inliers ratio = 0.83
RANSAC iteration 800: Inliers ratio = 0.70
RANSAC iteration 900: Inliers ratio = 0.46
RANSAC iteration 1000: Inliers ratio = 0.64
RANSAC iteration 1100: Inliers ratio = 0.41
RANSAC iteration 1200: Inliers ratio = 0.08
RANSAC iteration 1300: Inliers ratio = 0.00
RANSAC iteration 1400: Inliers ratio = 0.29
RANSAC iteration 1500: Inliers ratio = 0.00
RANSAC iteration 1600: Inliers ratio = 0.09
RANSAC iteration 1700: Inliers ratio = 0.67
RANSAC iteration 1800: Inliers ratio = 0.00
RANSAC iteration 1900: Inliers ratio = 0.68
RANSAC iteration 2000: Inliers ratio = 0.00
Homography matrix:
[[ 5.68341730e-01  2.13421760e-02  9.08598094e+02]
 [-1.88669363e-01  8.52307793e-01  1.70039992e+02]
 [-1.759